In [1]:
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
#from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.preprocessing import MinMaxScaler


In [3]:
data = pd.read_csv('PEP.csv')


In [4]:
data

,ticker,date,open,high,low,close
0,PEP,1972-06-01,0.000,1.60,1.580,1.60
1,PEP,1972-06-02,1.600,1.62,1.600,1.60
2,PEP,1972-06-05,1.600,1.60,1.540,1.57
3,PEP,1972-06-06,1.560,1.56,1.560,1.56
4,PEP,1972-06-07,1.560,1.56,1.550,1.55
...,...,...,...,...,...,...
12826,PEP,2023-04-13,182.620,184.61,182.105,184.38
12827,PEP,2023-04-14,183.335,184.02,182.920,183.51
12828,PEP,2023-04-17,184.190,184.75,183.481,184.45
12829,PEP,2023-04-18,184.770,185.22,183.740,184.83


In [5]:
data = data.filter(['close'])


In [6]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,close
11831,127.24
11832,126.83
11833,127.67
11834,127.27
11835,125.98
...,...
12826,184.38
12827,183.51
12828,184.45
12829,184.83


In [7]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [8]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [9]:

data = train_data

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]


In [10]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 7
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [11]:
start_time = time.time()


number_list = [5,6,8,9]

num_hidden_layer = number_list[0] # ví dụ chọn số 5

In [12]:

# Tạo mô hình CNN
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(size_window, 1)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
for i in range(num_hidden_layer): #Thêm num_hidden_layer lớp ẩn với 64 đơn vị mỗi lớp
    model.add(Dense(units=100, activation='relu')) #Tham số chỉ định số lượng tế bào thần kinh trong lớp
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

# Compile mô hình với Adam optimizer và Mean Squared Error loss
model.compile(loss='mse', optimizer='adam')


In [13]:
history = model.fit(X_train, y_train, epochs=11000, batch_size=8)

Epoch 1/9000
79/79 [==============================] - 2s 2ms/step - loss: 0.0502
Epoch 2/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0098
Epoch 3/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0056
Epoch 4/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0046
Epoch 5/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0043
Epoch 6/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 7/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 8/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0034
Epoch 9/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0030
Epoch 10/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 11/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 12/9000
79/79 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 13/9000
79/79 [====

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, (-1, 1))
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAPE: ', mape)

In [ ]:
len(data)